In [1]:
# !pip3 install git+https://github.com/rongardF/tvdatafeed.git@e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
from tvDatafeed import TvDatafeed, Interval

In [2]:
from helpers_db import get_engine, run_sql
engine = get_engine()

In [3]:
from helpers_queries import queries
assets = None
with engine.begin() as conn:
  assets = run_sql(conn, queries['list_assets_today']).mappings().all()
assets

[{'symbol': 'AAOI', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1},
 {'symbol': 'AAPL', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1},
 {'symbol': 'ALB', 'exchange': 'NYSE', 'category': 'stock', 'bars_d': 1},
 {'symbol': 'AMZN', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1},
 {'symbol': 'BABA', 'exchange': 'NYSE', 'category': 'stock', 'bars_d': 1},
 {'symbol': 'CI', 'exchange': 'NYSE', 'category': 'stock', 'bars_d': 1},
 {'symbol': 'CLS', 'exchange': 'NYSE', 'category': 'stock', 'bars_d': 1},
 {'symbol': 'CRDF', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1},
 {'symbol': 'CSCO', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1},
 {'symbol': 'DBK', 'exchange': 'XETR', 'category': 'stock', 'bars_d': 1},
 {'symbol': 'ESEA', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1},
 {'symbol': 'GCT', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1},
 {'symbol': 'GOOG', 'exchange': 'NASDAQ', 'category': 'stock', 'bars_d': 1},
 {'symbol': 'IN

In [4]:
from helpers_db import postgres_upsert

tv = TvDatafeed()

def fetch_ticks(symbol, exchange, interval=Interval.in_daily, bars=100):
  df = tv.get_hist(
      symbol=symbol,
      exchange=exchange,
      interval=interval,
      n_bars=bars,
      extended_session=True,
  )
  return df

def fix_symbol(eachange_symbol):
  return eachange_symbol.split(':')[-1] if ':' in eachange_symbol else eachange_symbol

def normalize_dataframe(df):
  df.reset_index(inplace=True)
  df['symbol'] = df['symbol'].apply(fix_symbol)
  if 'datetime' in list(df.columns.values):
    df['dt'] = df['datetime']
    df.drop(columns=['datetime'], inplace=True)
  return df

for asset in assets:
  symbol    = asset['symbol']
  exchange  = asset['exchange']
  bars      = 10
  interval  = Interval.in_daily
  print(f"Processing {exchange}:{symbol}")
  print(f"  count: {bars}")
  try:
    df        = fetch_ticks(symbol=symbol, exchange=exchange, bars=bars, interval=interval)
    df        = normalize_dataframe(df)
    df.to_sql('ticks_1d', engine, if_exists='append', index=False, method=postgres_upsert)
    
  except Exception as e:
    print(f"Exception {exchange}:{symbol}")
    print(str(e))
  # break


you are using nologin method, data you access may be limited


Processing NASDAQ:AAOI
  count: 10
Processing NASDAQ:AAPL
  count: 10
Processing NYSE:ALB
  count: 10
Processing NASDAQ:AMZN
  count: 10
Processing NYSE:BABA
  count: 10
Processing NYSE:CI
  count: 10
Processing NYSE:CLS
  count: 10
Processing NASDAQ:CRDF
  count: 10
Processing NASDAQ:CSCO
  count: 10
Processing XETR:DBK
  count: 10
Processing NASDAQ:ESEA
  count: 10
Processing NASDAQ:GCT
  count: 10
Processing NASDAQ:GOOG
  count: 10
Processing NASDAQ:INMD
  count: 10
Processing NYSE:JPM
  count: 10
Processing NYSE:KO
  count: 10
Processing EURONEXT:LQQ
  count: 10
Processing NASDAQ:META
  count: 10
Processing NASDAQ:MSFT
  count: 10
Processing NASDAQ:NKTX
  count: 10
Processing NASDAQ:NVDA
  count: 10
Processing NASDAQ:ON
  count: 10
Processing NYSE:PAYC
  count: 10
Processing NASDAQ:PYPL
  count: 10
Processing NASDAQ:SBUX
  count: 10
Processing NASDAQ:SEDG
  count: 10
Processing NYSE:T
  count: 10
Processing NASDAQ:WDAY
  count: 10
Processing NYSE:HEI
  count: 10
Processing NASDAQ:S